# Introduccion

El proyecto a desarrollar consiste en un sistema de vision artificial, que aprende a clasificar imagenes de animales. Para ello, entrenare a un modelo de Deep Learning con un conjunto de datos (imagenes) obtenido de una publicacion de Kaggle (para comprobar, acceder al siguiente enlace ==> https://www.kaggle.com/datasets/alessiocorrado99/animals10/data)


El proyecto se dividira principalmente en dos notebooks de Jupyter:

- El primero, enfocado al analisis y la visualizacion de las imagenes. Tambien se prepararan los datos para el entrenamiento del modelo.
- El segundo, enfocado a la creacion del modelo de Deep Learning, y a su entrenamiento.


Posteriormente, se desarrollara una aplicacion web a la que acceder a traves de una URL, que tendra conexion con una Base de Datos. Esta aplicacion web consistira en un formulario web donde el usuario podra subir una imagen, y se le devolvera un resultado (tras la comprobacion del modelo entrenado), y se registraran los datos generados en la BBDD.

# 1. Importar librerias y archivos necesarios

El primer paso para comenzar con el analisis de los datos, y su preparacion para el entrenamiento del modelo, es importar las librerias y archivos que vayamos a requerir.

In [ ]:
# Antes de ejecutar esta celda, por favor instalar dependencias del archivo 'requeriments.txt'.
# Es necesario instalar previamente las dependencias en el entorno que utilice la aplicacion.

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

# He cargado el dataset de imagenes en un zip en el directorio del notebook, por lo que debo descomprimirlo
import zipfile

In [ ]:
# Extraigo el contenido del archivo .zip

ruta_zip = 'dataset.zip'
ruta_extraccion = 'contenido_carpeta/' # Este directorio almacenara el contenido del archivo .zip
password = None

if not os.path.exists(ruta_extraccion):  # Si el directorio de extraccion no existe, se crea.
    
    os.mkdir(ruta_extraccion)

else:
    pass


archivo_zip = zipfile.ZipFile(ruta_zip, "r")
try:
    print(archivo_zip.namelist())  #Imprime el nombre de los archivos, que es un directorio y un archivo con extension .py
    
    # Reviso que no exista antes el directorio descomprimido
    if not os.path.exists('contenido_carpeta/raw-img/'):
        archivo_zip.extractall(pwd=password, path=ruta_extraccion)
        print('El .zip ha sido descomprimido con exito.')
    else:
        print('La ruta objetivo ya contiene los archivos del .zip .')
except:
    pass
archivo_zip.close()




In [ ]:
# Revisamos el contenido que almacenaba el .zip

for archivo in os.listdir(ruta_extraccion):
    if os.path.isdir(os.path.join(ruta_extraccion, archivo)):
        print(archivo, '==> Directorio')
    else:
        print(archivo)

In [ ]:
# Podemos ver como el .zip contiene una carpeta llamada "raw-img", y un script en Python.
# Si revisamos el archivo .py, veremos que el contenido es el siguiente:

translate = {"cane": "dog",
             "cavallo": "horse",
             "elefante": "elephant",
             "farfalla": "butterfly",
             "gallina": "chicken",
             "gatto": "cat",
             "mucca": "cow",
             "pecora": "sheep",
             "scoiattolo": "squirrel", 
             "dog": "cane",
             "cavallo": "horse",
             "elephant" : "elefante",
             "butterfly": "farfalla",
             "chicken": "gallina",
             "cat": "gatto",
             "cow": "mucca",
             "spider": "ragno",
             "squirrel": "scoiattolo"}  # DIccionario copiado del archivo .py


# Se puede ver como los pares clave-valor del diccionario contienen las traducciones de los nombres de los animales.
# Estas traducciones podemos leerlas de la siguiente manera:
#  - italiano ==> ingles
#  - ingles ==> italiano

In [ ]:
# A continuacion, muestro el contenido del directorio 'raw-img/'
ruta_directorio_imagenes = 'contenido_carpeta/raw-img/'

for archivo in os.listdir(ruta_directorio_imagenes):
    if os.path.isdir(os.path.join(ruta_directorio_imagenes, archivo)):
        print(archivo, '==> Directorio')
    else:
        print(archivo, 'No es un Directorio')

In [ ]:
# En este caso, podemos observar como el contenido del directorio esta conformado por nuevas carpetas, que contienen las
# imagenes de cada uno de los animales que el modelo a entrenar debera ser capaz de clasificar.

# Antes de continuar, voy a modificar los nombres de estos directorios a su respectiva version en ingles, haciendo uso
# del diccionario que estaba definido en el archivo .py .

for archivo in os.listdir(ruta_directorio_imagenes):
    if archivo in translate.keys():
        try:
            os.rename(os.path.join(ruta_directorio_imagenes, archivo), os.path.join(ruta_directorio_imagenes, translate[archivo]))
            print('Directorio {} ==> {}'.format(os.path.join(ruta_directorio_imagenes, archivo), os.path.join(ruta_directorio_imagenes, translate[archivo])))
        except FileExistsError:
            print('El directorio {} ya ha sido renombrado con anterioridad.'.format(archivo))

In [ ]:
# Para continuar, veo apropiado crear un nuevo directorio al que destinar las carpetas con las imagenes de los animales.

nuevo_directorio = 'datasets_animales/'

# Implemento la creacion del directorio mediante un condicional para evitar posibles errores al ejecutar de nuevo el notebook.

if not os.path.exists(nuevo_directorio):
    os.makedirs(nuevo_directorio)
    print('Directorio {} creado con exito.'.format(nuevo_directorio))
else:
    print('El directorio "{}" ya existe.'.format(nuevo_directorio))  

In [ ]:
# Una vez creado el directorio, muevo las carpetas con las imagenes de los animales a esta ubicacion.

import shutil  # ==> libreria que me permite copiar archivos a nuevas ubicaciones
from tqdm import tqdm # Esta libreria es opcional; mejora la visualizacion de los procesos que requieren iteraciones.

for carpeta in tqdm(os.listdir(ruta_directorio_imagenes)):
        
    if not os.path.exists(os.path.join(nuevo_directorio, carpeta)):
        # Creo un directorio con el mismo nombre dentro de la ruta destino
        os.makedirs(nuevo_directorio + carpeta)
        ruta_carpeta_animal = 'datasets_animales/{}/'.format(carpeta)
        for imagen in os.listdir(os.path.join(ruta_directorio_imagenes, carpeta)):
            shutil.copyfile(os.path.join(ruta_directorio_imagenes + carpeta, imagen), os.path.join(ruta_carpeta_animal, imagen))
        else: 
            print('Contenido de la carpeta {} trasladado con exito.'.format(os.path.join(ruta_directorio_imagenes, carpeta)))
    else:
        print('El directorio {} ya existe.'.format(os.path.join(nuevo_directorio, carpeta)))

# 2. Analisis del conjunto de datos 

Este proceso es muy importante, ya que sera util para revisar y comprender nuestro conjunto de datos.
En este proyecto, el conjunto de datos esta compuesto por imagenes, por lo que realmente, no podemos obtener mucha mas 
informacion que la que vemos en la propia imagen (en la creacion de otro tipo de modelos, es posible que analizar los
datos contenidos en ficheros Excel o .csv nos ayuden a identificar patrones en los datos, permitiendonos un mejor preprocesado de los mismos antes de lanzarlos al modelo).

In [ ]:
# A continuacion, veremos cuantas imagenes existen en cada directorio ==> cuantos ejemplos tenemos de cada etiqueta
# para entrenar nuestro modelo.

for carpeta in tqdm(os.listdir(nuevo_directorio)):
    n_ejemplos = len(os.listdir(os.path.join(nuevo_directorio, carpeta)))
    print('El conjunto de datos contiene {} ejemplos para la etiqueta {}.'.format(n_ejemplos, carpeta))

In [ ]:
# La instruccion nos muestra como NO tenemos el mismo numero de ejemplos para cada etiqueta.
# Una solucion que podremos aplicar a esto es generar nuevas imagenes con Data Augmentation, obteniendo un numero
# similar de ejemplos para todas las etiquetas.

In [ ]:
print(os.getcwd())

In [ ]:
# En cuanto a las imagenes, voy a visualizar un grupo de estas para una especie en concreto, y revisar si todas tienen
# la misma resolucion.

# Muestro 5 imagenes de la carpeta 'butterfly':
contador = 0

for imagen in os.listdir('datasets_animales/butterfly/'):
    if contador < 5:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/butterfly/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2RGB)
        contador +=1
        
        plt.imshow(imagenRGB)
        plt.show()
    else: 
        break


In [ ]:
# Estas 5 primeras imagenes me permiten observar dos factores a tener en cuenta:
#  - Las imagenes no tienen todas la misma resolucion, por lo que debere aplicar algun preprocesado para 
#    asignar unas dimensiones comunes para todas las imagenes.

#  - Como existen individuos de diferentes colores para una misma especie (al igual que ocurre con el fondo de las
#    imagenes, cargare las imagenes en escala de grises para el entrenamiento del modelo). Esto permitira simplificar el
#    modelo, a la vez que tratar de influir positivamente en el proceso de aprendizaje de la red.

In [ ]:
# Voy a mostrar estas mismas imagenes en escala de grises:

contador = 0

for imagen in os.listdir(os.path.join('datasets_animales/butterfly/')):
    if contador < 5:
        imagenBGR = cv2.imread(os.path.join('datasets_animales/butterfly/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenBGR, cv2.COLOR_BGR2GRAY)
        contador +=1
        
        plt.imshow(imagenRGB, cmap='gray')
        plt.show()
    else: 
        break

In [ ]:
# Muestro las 5 primeras imagenes de otro animal, en color y en escala de grises

contador = 0

for imagen in os.listdir(os.path.join('datasets_animales/dog/')):
    if contador < 5:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/dog/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2RGB)
        contador +=1
        
        plt.imshow(imagenRGB)
        plt.show()
    else: 
        break


In [ ]:
# Visualizo las mismas imagenes, en escala de grises

contador = 0

for imagen in os.listdir(os.path.join('datasets_animales/dog/')):
    if contador < 5:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/dog/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_RGB2GRAY)
        contador +=1
        
        plt.imshow(imagenRGB, cmap='gray')
        plt.show()
    else: 
        break


In [ ]:
# Realmente, las imagenes no tienen la mejor resolucion del mundo, pero serviran para comprender el proceso de construccion
# y entrenamiento de este tipo de modelos

In [ ]:
# Reviso la resolucion de las 5 primeras imagenes que hemos visto. 
# En este caso, corresponden a las imagenes de mariposas.

contador = 0

for imagen in os.listdir(os.path.join('datasets_animales/butterfly/')):
    if contador < 5:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/butterfly/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2RGB)
        contador +=1
        
        plt.imshow(imagenRGB)
        plt.show()
        print('Resolucion de la imagen ==>', imagenRGB.shape)
    else: 
        break


In [ ]:
# Hago lo mismo con las imagenes de los perros

contador = 0

for imagen in os.listdir(os.path.join('datasets_animales/dog/')):
    if contador < 5:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/dog/', imagen))
        
        # Cargo la imagen con gama cromatica RGB
        imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2RGB)
        contador +=1
        
        plt.imshow(imagenRGB)
        plt.show()
        
        print('Resolucion de la imagen ==>', imagenRGB.shape)
    else: 
        break


In [ ]:
# En este caso (perros), vemos que hay mas imagenes que coinciden en la resolucion. Por ello, pienso que seria correcto asignar una resolucion de
# 225x300 a todas las imagenes (el ultimo elemento de la tupla , 3, hace referencia a los canales de color, que en el caso de las imagenes con
# escala de grises que utilizare, sera 1).

In [ ]:
# A fin de no crear nuevas imagenes, y evitar cargar mas datos, voy a modificar las imagenes del propio directorio de 'datasets_animales'.

# Si la imagen tiene mas de 300px de ancho, volteare la imagen.
# Posteriormente, independientemente de si he volteado o no la imagen, le asignare a todas unas dimensiones de 225x300. Probare con esta resolucion,
# pero tal vez es demasiado, provocando un aumento en la complejidad de la red neuronal, con la consecuente demora en el entrenamiento.


# FUNCION PARA ROTAR IMAGENES
# Esta funcion devuelve la imagen modificada, aplicandole un tono de color en escala de grises.

def rotate(image, angle, center=None, scale=1.0):
    """
    Rota la imagen de entrada alrededor de un punto específico, dado un cierto ángulo.
    :param image: Imagen a ser rotada.
    :param angle: Ángulo de rotación.  ==> Este puede ser positivo, giro antihorario; o negativo, giro horario
    :param center: Ponto alrededor del cual rotar.
    :param scale: Escala de la imagen resultante.
    :return: Imagen rotada y estirada para ocupar los bordes.
    """
    # Extraemos las dimensiones de la imagen.
    (h, w) = image.shape[:2]

    # Redimensionamos la imagen antes de la rotación.
    resized_image = cv2.resize(image, (int(w * scale), int(h * scale)))

    # Por defecto, estableceremos el punto de rotación en el centro de la imagen redimensionada.
    if center is None:
        center = (resized_image.shape[1] // 2, resized_image.shape[0] // 2)

    # Calculamos la matriz de rotación con base al centro, ángulo y escala.
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Llevamos a cabo la rotación.
    rotated = cv2.warpAffine(resized_image, rotation_matrix, (resized_image.shape[1], resized_image.shape[0]))

    return rotated



In [ ]:
#os.chdir('..')

In [ ]:
#for carpeta in os.listdir():
#    print(carpeta)

In [ ]:
print('Directorio antes ==>', os.getcwd())
os.chdir(nuevo_directorio)
print('Directorio actual ==>', os.getcwd())

for carpeta in os.listdir():
    
    os.chdir('{}/'.format(carpeta))  # Cada vez que se itere sobre una nueva carpeta, la marco como el directorio actual. 
    print('Directorio actual (dentro animal) ==>', os.getcwd())
    
    for imagen in tqdm(os.listdir()):
        #print('nombre ==> ', imagen)
        #print(os.path.join('datasets_animales/{}/'.format(carpeta), imagen))
        imagenBGR = cv2.imread(imagen)

        # Compruebo la resolucion de la imagen
        resolucion_imagen = imagenBGR.shape
        #print(resolucion_imagen)
        
        if resolucion_imagen[0] > 400: # La resolucion de la imagen viene dada en una tupla de la forma (alto, ancho, canales de color)
            # Rotamos la imagen
            #imagenRotada = rotate(image = imagenBGR, angle = 90)
            imagenRedimensionada = cv2.resize(imagenBGR, dsize=(120, 160), interpolation=cv2.INTER_CUBIC)
            imagenRotada = cv2.rotate(imagenRedimensionada, cv2.ROTATE_90_CLOCKWISE)
    
            # Convertimos la imagen a escala de grises
            imagenFinal = cv2.cvtColor(imagenRotada, cv2.COLOR_RGB2GRAY)
            
            try:
            
                # Guardamos la imagen rotada
                cv2.imwrite(imagen, imagenFinal)  # Asigno el nombre original a la imagen, para evitar crear una copia de cada una.
            
            except Exception as e:
                
                try:
                
                    cv2.imwrite(imagen, imagenFinal)
                    
                except Exception as e:
                        
                    print('Ha ocurrido un error al tratar de leer la imagen ==>', e)
            
            contador +=1
        
        else:
            # En caso de que no sea necesario rotar la imagen, solamente le aplico el cambio de resolucion, y le asigno una escala de grises
            imagenFinal = cv2.resize(imagenBGR, dsize=(120, 160), interpolation=cv2.INTER_CUBIC)

            # Convertimos la imagen a escala de grises
            imagenFinal = cv2.cvtColor(imagenFinal, cv2.COLOR_RGB2GRAY)
            
            # El siguiente bloque try-except sera util para evitar el error ==>  (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
            try:
            
                # Guardamos la imagen rotada
                cv2.imwrite(imagen, imagenFinal)
            
            except Exception as e:
                
                try:
                
                    cv2.imwrite(imagen, imagenFinal)
                    
                except Exception as e:
                    
                    print('Ha ocurrido un error al tratar de leer la imagen ==>', e)
                
            contador +=1
    
    else:  # Una vez se haya recorrido el contenido de la carpeta para un animal en concreto, cambio el directorio en el que opera Python para evitar errores
        # Obtenemos el directorio padre del directorio actual
        os.chdir('..')
        os.chdir('..')
        os.chdir(nuevo_directorio)
            


In [ ]:
os.chdir('..')

for carpeta in os.listdir(nuevo_directorio):
    print('Existen {} ejemplos de {} pare el entrenamiento de la red.'.format(len(os.listdir(os.path.join(nuevo_directorio, carpeta))), carpeta))

# 3. Visualizacion del conjunto de datos

Un proceso importante en la creacion de modelos de Aprendizaje Automatico y Deep Learning, consiste en la visualizacion de los datos de los que disponemos. 

En casos en los que se tienen datos numericos/categoricos, este proceso ayuda a obtener una vision general de los datos, y permite facilitar la comprension del objetivo que tratamos de conseguir con la creacion y entrenamiento de un modelo.

In [ ]:
# En mi caso, dispongo unicamente de imagenes, por lo que voy a mostrar un ejemplo de las que tenemos tras aplicar
# el proceso de normalizacion.

# Creo un array donde meter las primeras 20 imagenes para el caso de 'butterfly'

imagenes = []

for imagen in os.listdir('datasets_animales/butterfly/'):
    
    if len(imagenes) == 20:  # Una vez carguemos 20 imagenes, rompemos el bucle para continuar con la visualizacion de estas.
        break
    else:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/butterfly/', imagen))
        #imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2GRAY)
        imagenes.append(imagenRGB)
        
        
for imagen in imagenes:
    print('Resolucion de la imagen ==> ', imagen.shape)

        


# Configuramos el diseño de la figura
filas = 4
columnas = 5

# Creamos una figura y un conjunto de ejes con el tamaño adecuado para mostrar 20 imágenes
fig, axs = plt.subplots(filas, columnas, figsize=(12, 10))

# Iteramos sobre las filas y columnas para mostrar las imágenes
for i in range(filas):
    for j in range(columnas):
        # Calculamos el índice de la imagen actual en la lista de imágenes
        indice_imagen = i * columnas + j
        
        # Mostramos la imagen si está dentro de la lista de imágenes
        if indice_imagen < len(imagenes):
            axs[i, j].imshow(imagenes[indice_imagen], cmap='gray')  # Mostramos imágenes en escala de grises
            axs[i, j].axis('off')  # Ocultamos los ejes para una apariencia más limpia
        else:
            axs[i, j].axis('off')  # Si no hay más imágenes, ocultamos la celda
        
        # Agregamos un título a las imágenes (opcional)
        axs[i, j].set_title(f'Imagen {indice_imagen + 1}', fontsize=10)

# Ajustamos el espacio entre las imágenes para que se vean mejor
plt.tight_layout()

# Mostramos la figura con las imágenes
plt.show()

In [ ]:
# Se puede apreciar como algunas imagenes estan al reves. Debido a la logica aplicada, para lograr una correcta redimension de 
# estas imagenes, en el caso de que una tuviese un altura superior a 400px, la imagen se voltaa para asignar esta altura como ancho,
# y viceversa.
# Tras esto, se redimensiona la imagen a 120x160, evitando que la imagen se estreche mas de lo conveniente. 

In [ ]:
# Al parecer, la dimension de algunas de estas imagenes (butterfly) se han asignado al reves,225x300.
# Voy a recorrer todas las imagenes de esta carpeta, y en caso de que la resolucion de una imagen sea 225x300, la rotare.

# Directorio de las imágenes
directorio = 'datasets_animales/butterfly/'

# Iterar sobre las imágenes en el directorio
for imagen in tqdm(os.listdir(directorio)):
    # Obtener la ruta completa de la imagen
    ruta_imagen = os.path.join(directorio, imagen)
    
    # Cargar la imagen
    imagenCargada = cv2.imread(ruta_imagen)
    
    # Verificar las dimensiones de la imagen
    resolucion_imagen = imagenCargada.shape
    
    # Si la imagen tiene una altura de 120 píxeles
    if resolucion_imagen[1] == 160:
        try:
            # Rotar la imagen 90 grados en sentido horario
            imagenRotada = cv2.rotate(imagenCargada, cv2.ROTATE_90_CLOCKWISE)
            
            # Guardar la imagen rotada
            cv2.imwrite(os.path.join('datasets_animales/butterfly/', imagen), imagenRotada)
            
        except Exception as e:
            print('Ha ocurrido un error al girar la imagen {} ==> {}'.format(imagen, e))

  
print('Se han girado correctamente las imagenes.')

In [ ]:
imagenes = []

for imagen in os.listdir('datasets_animales/cat/'):
    
    if len(imagenes) == 20:  # Una vez carguemos 20 imagenes, rompemos el bucle para continuar con la visualizacion de estas.
        break
    else:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/cat/', imagen))
        #imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2GRAY)
        imagenes.append(imagenRGB)
        
        
for imagen in imagenes:
    print('Resolucion de la imagen ==> ', imagen.shape)

        


# Configuramos el diseño de la figura
filas = 4
columnas = 5

# Creamos una figura y un conjunto de ejes con el tamaño adecuado para mostrar 20 imágenes
fig, axs = plt.subplots(filas, columnas, figsize=(12, 10))

# Iteramos sobre las filas y columnas para mostrar las imágenes
for i in range(filas):
    for j in range(columnas):
        # Calculamos el índice de la imagen actual en la lista de imágenes
        indice_imagen = i * columnas + j
        
        # Mostramos la imagen si está dentro de la lista de imágenes
        if indice_imagen < len(imagenes):
            axs[i, j].imshow(imagenes[indice_imagen], cmap='gray')  # Mostramos imágenes en escala de grises
            axs[i, j].axis('off')  # Ocultamos los ejes para una apariencia más limpia
        else:
            axs[i, j].axis('off')  # Si no hay más imágenes, ocultamos la celda
        
        # Agregamos un título a las imágenes (opcional)
        axs[i, j].set_title(f'Imagen {indice_imagen + 1}', fontsize=10)

# Ajustamos el espacio entre las imágenes para que se vean mejor
plt.tight_layout()

# Mostramos la figura con las imágenes
plt.show()

In [ ]:
imagenes = []

for imagen in os.listdir('datasets_animales/horse/'):
    
    if len(imagenes) == 20:  # Una vez carguemos 20 imagenes, rompemos el bucle para continuar con la visualizacion de estas.
        break
    else:
        imagenRGB = cv2.imread(os.path.join('datasets_animales/horse/', imagen))
        #imagenRGB = cv2.cvtColor(imagenRGB, cv2.COLOR_BGR2GRAY)
        imagenes.append(imagenRGB)
        
        
for imagen in imagenes:
    print('Resolucion de la imagen ==> ', imagen.shape)

        


# Configuramos el diseño de la figura
filas = 4
columnas = 5

# Creamos una figura y un conjunto de ejes con el tamaño adecuado para mostrar 20 imágenes
fig, axs = plt.subplots(filas, columnas, figsize=(12, 10))

# Iteramos sobre las filas y columnas para mostrar las imágenes
for i in range(filas):
    for j in range(columnas):
        # Calculamos el índice de la imagen actual en la lista de imágenes
        indice_imagen = i * columnas + j
        
        # Mostramos la imagen si está dentro de la lista de imágenes
        if indice_imagen < len(imagenes):
            axs[i, j].imshow(imagenes[indice_imagen], cmap='gray')  # Mostramos imágenes en escala de grises
            axs[i, j].axis('off')  # Ocultamos los ejes para una apariencia más limpia
        else:
            axs[i, j].axis('off')  # Si no hay más imágenes, ocultamos la celda
        
        # Agregamos un título a las imágenes (opcional)
        axs[i, j].set_title(f'Imagen {indice_imagen + 1}', fontsize=10)

# Ajustamos el espacio entre las imágenes para que se vean mejor
plt.tight_layout()

# Mostramos la figura con las imágenes
plt.show()

In [ ]:
# Se puede apreciar que, al menos en las imagenes mostradas para el resto de animales, las imagenes se mantienen en la proporcion correcta.

# Sin embargo, para prevenir cualquier tipo de error futuro al tratar de alimentar a la red neuronal con los datos, voy a iterar sobre TODAS las imagenes
# de nuestro dataset, y girare aquellas que tengan resolucion 225x300.

directorio = 'datasets_animales/'
contadorImagenes = 0

for carpeta in tqdm(os.listdir(directorio)):
    
    if carpeta == 'butterfly':  # Como ya he revisado los ejemplos para este animal, me ahorro el proceso de iteracion.
        continue
    else:
        for imagen in os.listdir(os.path.join(directorio, carpeta)):
            
            # Obtener la ruta completa de la imagen
            ruta_imagen = os.path.join(directorio + carpeta + '/', imagen)
            
            # Cargar la imagen
            imagenCargada = cv2.imread(ruta_imagen)
            
            # Verificar las dimensiones de la imagen
            resolucion_imagen = imagenCargada.shape
            
            # Si la imagen tiene una altura de 120 píxeles
            if resolucion_imagen[1] == 160:
                contadorImagenes +=1
                try:
                    # Rotar la imagen 90 grados en sentido horario
                    imagenRotada = cv2.rotate(imagenCargada, cv2.ROTATE_90_CLOCKWISE)
                    
                    # Guardar la imagen rotada
                    cv2.imwrite(os.path.join(directorio + carpeta + '/', imagen), imagenRotada)
                    
                except Exception as e:
                    print('Ha ocurrido un error al girar la imagen {} ==> {}'.format(imagen, e))

            
print('Se han girado correctamente un total de {} imagenes.'.format(contadorImagenes))
        

Tras esto, doy por finalizado el proceso de analisis y preprocesamiento de las imagenes para el entrenamiento del modelo.

Continuare con el desarrollo en el archivo 'Preparacion datos/creacion y entrenamiento Modelo Clasificacion.ipynb'.